__UNet - perception loss in fastai v2 - Basemodel_Hagen__

In [ ]:
from fastai.vision.all import *
import os
import pathlib
import fastai; fastai.__version__

In [ ]:
torch.cuda.set_device(0)

In [ ]:
torch.cuda.current_device(), torch.cuda.get_device_name(0)

In [ ]:
# set to torch.device("cpu"), if you do not have a graphic card to work with!
device=torch.device("cuda")

In [ ]:
path = pathlib.Path(os.getcwd())

In [ ]:
path_GT = path/'Daten/Hagen_GT'
path_noisy = path/'Daten/Hagen_noisy'

"""
# use the following path for Actin data only
path_GT = path/'Daten/Hagen_actin_GT'
path_noisy = path/'Daten/Hagen__noisy'
"""

In [ ]:
# set seeds for everything for reproducibility
%run utility/set_seeds.ipynb

In [ ]:
# run Dataloader
%run utility/Dataloader_Hagen.ipynb

In [ ]:
# run notebook to set loss 
%run utility/Perception_loss.ipynb

In [ ]:
# apply dataloader with batchsize of 38 for RTX4060, 16GB - loss weights may need be tuned for other bs
dls_den, dblock = get_dls(16)

In [ ]:
# setup loss function and weights for indiviual loss components - approximately equal loss in first epoch
#loss_func = FeatureLoss(vgg_m, blocks[2:5],[100], [5,15,17], [3,22,21])    # aedequate weights for actin only
loss_func = FeatureLoss(vgg_m, blocks[2:5],[100], [6,16,19], [4,25,22])    # tuned weights for all Hagen data

In [ ]:
# set metrics to track individual loss components 
metrics = LossMetrics(loss_func.metric_names)

In [ ]:
# set file name and tracking information to store training metrics
fname = 'Basemodel_Hagen_all'
cbs = CSVLogger(fname=fname+'.csv')                                                

In [ ]:
# define model architecture and parameters, pretrained false sets all layers as trainable
def create_gen_learner():
    return unet_learner(dls_den, bbone, loss_func=loss_func, blur=True, norm_type=NormType.Weight, 
                        self_attention=True, y_range=y_range, pretrained=False, weights=None,
                        metrics=metrics, cbs=cbs
                        )

In [ ]:
# create denoising learner
learn_den = create_gen_learner()
learn_den.to_fp16()
learn_den.summary() # to check that all layers are trainable

In [ ]:
# learning rate finder to set proper learning rate - high slope for loss
learn_den.lr_find()

In [ ]:
# training using the one cycle policy - learning rate scheduler
learn_den.fit_one_cycle(200,1e-3,wd=wd)

In [ ]:
# save model
learn_den.save(fname)